In [10]:
import pandas as pd
import numpy as np
import sklearn
from jupyter_core.migrate import regex
from sklearn.model_selection import train_test_split,GridSearchCV,KFold,StratifiedKFold
from sklearn.preprocessing import *
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.metrics import *
from sklearn.compose import *
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier,AdaBoostRegressor,GradientBoostingRegressor

In [29]:
df=pd.read_csv(r"C:\Users\dai\Downloads\assigment data\train.csv",index_col=0)
df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
Item_Identifier,,,,,,,,,,,
FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [30]:
df=df.drop(['Outlet_Identifier','Outlet_Establishment_Year'],axis=1)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8523 entries, FDA15 to DRG01
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Item_Weight           7060 non-null   float64
 1   Item_Fat_Content      8523 non-null   object 
 2   Item_Visibility       8523 non-null   float64
 3   Item_Type             8523 non-null   object 
 4   Item_MRP              8523 non-null   float64
 5   Outlet_Size           6113 non-null   object 
 6   Outlet_Location_Type  8523 non-null   object 
 7   Outlet_Type           8523 non-null   object 
 8   Item_Outlet_Sales     8523 non-null   float64
dtypes: float64(4), object(5)
memory usage: 665.9+ KB


In [32]:
df.isnull().sum()

Item_Weight             1463
Item_Fat_Content           0
Item_Visibility            0
Item_Type                  0
Item_MRP                   0
Outlet_Size             2410
Outlet_Location_Type       0
Outlet_Type                0
Item_Outlet_Sales          0
dtype: int64

In [33]:
df.Item_Fat_Content.unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [34]:
df['Item_Fat_Content']=df['Item_Fat_Content'].replace({'LF':"low fat",
                                                       "Low Fat": 'low fat',
                                                       "reg":"regular",
                                                       "Regular":'regular',
                                                       },regex=True)

In [35]:
df.Item_Fat_Content.unique()

array(['low fat', 'regular'], dtype=object)

In [43]:
df.groupby(['Item_Type','Item_Fat_Content'])['Item_Weight'].transform(lambda x : x.mean())

Item_Identifier
FDA15    13.391497
DRC01    11.228220
FDN15    13.565644
FDX07    13.708363
NCD19    13.384736
           ...    
FDF22    13.707177
FDS36    12.013303
NCJ29    13.142314
FDN46    12.027116
DRG01    11.963444
Name: Item_Weight, Length: 8523, dtype: float64